In [1]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce
import os
import re

In [2]:
def split_time(df):
    sub = df['patient_id'].split('-')
    try:
        time = int(sub[2])
        return time
    except IndexError:
        return 0

In [3]:
def remove_time(df):
    patient = df['patient_id'].split('-')
    return '{}-{}'.format(patient[0], patient[1])

In [4]:
def remove_day(x):
    if isinstance(x, str):
        x = x.replace('day ', '')
    return x

In [5]:
add_info = pd.read_csv('C:/Users/lzoeckler/Desktop/namibia_PCR/formatted_namibia_PCR.csv')
add_info.head()

,participant_id,age_yrs,sex,day0_enrollee,days_since_detection_enroll,time_to_negativity_urdt,where_detected,timepoint_days,quansys_HRP2_pg_ml,qpcr_p_density,HRP2_result,qpcr_result
0,pa-001,2,Female,1,0,29,Health Facility,3,318702.59,NaN,1.0,NaN
1,pa-001,2,Female,1,0,29,Health Facility,7,73737.79,NaN,1.0,NaN
2,pa-001,2,Female,1,0,29,Health Facility,14,4138.79,NaN,1.0,0.0
3,pa-001,2,Female,1,0,29,Health Facility,21,1377.18,NaN,1.0,NaN
4,pa-002,80,Female,1,0,35,Health Facility,0,43928722.05,NaN,NaN,NaN


In [6]:
add_info.drop_duplicates(subset=['participant_id', 'timepoint_days'], inplace=True, keep='last')
add_info.head()

,participant_id,age_yrs,sex,day0_enrollee,days_since_detection_enroll,time_to_negativity_urdt,where_detected,timepoint_days,quansys_HRP2_pg_ml,qpcr_p_density,HRP2_result,qpcr_result
0,pa-001,2,Female,1,0,29,Health Facility,3,318702.59,NaN,1.0,NaN
1,pa-001,2,Female,1,0,29,Health Facility,7,73737.79,NaN,1.0,NaN
2,pa-001,2,Female,1,0,29,Health Facility,14,4138.79,NaN,1.0,0.0
3,pa-001,2,Female,1,0,29,Health Facility,21,1377.18,NaN,1.0,NaN
4,pa-002,80,Female,1,0,35,Health Facility,0,43928722.05,NaN,NaN,NaN


In [7]:
time_df = pd.read_csv('C:/Users/lzoeckler/Desktop/all_data/separated/18-UCSF-01.csv')
time_df = time_df[['participant_id', 'timepoint_days', 'age_yrs', 'quansys_HRP2_pg_ml']]
time_df = time_df.merge(add_info, how='inner', on=['participant_id', 'timepoint_days'])
time_df.sort_values(['participant_id', 'timepoint_days'], inplace=True)
time_df.set_index(['participant_id', 'timepoint_days'], inplace=True)
time_df.head()

age_yrs_x quansys_HRP2_pg_ml_x  age_yrs_y  \
participant_id timepoint_days                                              
pa-001         3                       2            318702.59          2   
               7                       2             73737.79          2   
               14                      2              4138.79          2   
               21                      2              1377.18          2   
pa-002         0                      80          43928722.05         80   

                                  sex  day0_enrollee  \
participant_id timepoint_days                          
pa-001         3               Female              1   
               7               Female              1   
               14              Female              1   
               21              Female              1   
pa-002         0               Female              1   

                               days_since_detection_enroll  \
participant_id timepoint_days                                
pa-001         3                                         0   
               7                                         0   
               14                                        0   
               21                                        0   
pa-002         0                                         0   

                               time_to_negativity_urdt   where_detected  \
participant_id timepoint_days                                             
pa-001         3                                    29  Health Facility   
               7                                    29  Health Facility   
               14                                   29  Health Facility   
               21                                   29  Health Facility   
pa-002         0                                    35  Health Facility   

                              quansys_HRP2_pg_ml_y  qpcr_p_density  \
participant_id timepoint_days                                        
pa-001         3                         318702.59             NaN   
               7                          73737.79             NaN   
               14                          4138.79             NaN   
               21                          1377.18             NaN   
pa-002         0                       43928722.05             NaN   

                               HRP2_result  qpcr_result  
participant_id timepoint_days                            
pa-001         3                       1.0          NaN  
               7                       1.0          NaN  
               14                      1.0          0.0  
               21                      1.0          NaN  
pa-002         0                       NaN          NaN

In [8]:
check = time_df.reset_index()
len(check)

723

In [10]:
check.head()

,participant_id,timepoint_days,age_yrs_x,quansys_HRP2_pg_ml_x,age_yrs_y,sex,day0_enrollee,days_since_detection_enroll,time_to_negativity_urdt,where_detected,quansys_HRP2_pg_ml_y,qpcr_p_density,HRP2_result,qpcr_result
0,pa-001,3,2,318702.59,2,Female,1,0,29,Health Facility,318702.59,NaN,1.0,NaN
1,pa-001,7,2,73737.79,2,Female,1,0,29,Health Facility,73737.79,NaN,1.0,NaN
2,pa-001,14,2,4138.79,2,Female,1,0,29,Health Facility,4138.79,NaN,1.0,0.0
3,pa-001,21,2,1377.18,2,Female,1,0,29,Health Facility,1377.18,NaN,1.0,NaN
4,pa-002,0,80,43928722.05,80,Female,1,0,35,Health Facility,43928722.05,NaN,NaN,NaN


In [9]:
check2 = pd.read_csv('C:/Users/lzoeckler/Desktop/all_data/separated/18-UCSF-01.csv')
len(check2)

1522

In [11]:
check2.head()

,Unnamed: 0,study_id,sample_id,participant_id,sample_type,country,lat,long,age_yrs,timepoint_days,...,quansys_LDH_Pan_pg_ml,quansys_LDH_Pv_pg_ml,quansys_LDH_Pf_pg_ml,quansys_CRP_ng_ml,quansys_HRP2_result,quansys_LDH_Pan_result,quansys_LDH_Pv_result,quansys_LDH_Pf_result,quansys_CRP_result,quansys_type
0,7772,18-UCSF-01,pa-001-14,pa-001,WB,Namibia ...,NaN,NaN,2,14,...,< 14.41,2.76,NaN,7064.05,NaN,NaN,NaN,NaN,NaN,4plex
1,7773,18-UCSF-01,pa-001-21,pa-001,WB,Namibia ...,NaN,NaN,2,21,...,38.94,12.21,NaN,2413.65,NaN,NaN,NaN,NaN,NaN,4plex
2,7774,18-UCSF-01,pa-001-28,pa-001,WB,Namibia ...,NaN,NaN,2,28,...,29.86,13.9,NaN,92.87,NaN,NaN,NaN,NaN,NaN,4plex
3,7775,18-UCSF-01,pa-001-3,pa-001,WB,Namibia ...,NaN,NaN,2,3,...,142.3,15.35,NaN,5678.58,NaN,NaN,NaN,NaN,NaN,4plex
4,7776,18-UCSF-01,pa-001-7,pa-001,WB,Namibia ...,NaN,NaN,2,7,...,179.34,10.31,NaN,1021.09,NaN,NaN,NaN,NaN,NaN,4plex


In [141]:
time_df.to_csv('C:/Users/lzoeckler/Desktop/4plex/output_data/final_dilutions.csv')

In [142]:
partial_format = samples_data.copy(deep=True)
partial_format['time_point_days'] = partial_format.apply(split_time, axis=1)
partial_format['patient_id'] = partial_format.apply(remove_time, axis=1)
partial_format = partial_format.merge(add_info, how='left', on=['patient_id', 'time_point_days'])
partial_format.sort_values(['patient_id', 'time_point_days'], inplace=True)
partial_format.set_index(['patient_id', 'time_point_days'], inplace=True)
partial_format.head()

well error     HRP2_pg_ml    LDH_Pan_pg_ml  \
patient_id time_point_days                                              
pa-001     3                 g6   NaN         145.95           142.30   
           3                 f3   NaN      397468.45     < 1801250.00   
           3                 e3   NaN      318702.59         74360.14   
           3                 h3   NaN  1537195777.01  119019730410.15   
           3                 h6   NaN       10273.61          2535.11   

                              LDH_Pv_pg_ml        CRP_ng_ml concentration  \
patient_id time_point_days                                                  
pa-001     3                         15.35          5678.58             1   
           3                   < 291250.00     < 3342500.00        125000   
           3                      10385.14       < 66850.00          2500   
           3                16185275422.09  286153421264.94     312500000   
           3                        733.44         49491.76            50   

                            day0_enrollee  days_since_tx  initial_sample  \
patient_id time_point_days                                                 
pa-001     3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   

                            got_PQ  age     sex  fever48_r    enrolled_from  \
patient_id time_point_days                                                    
pa-001     3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   

                            returned_with_fever when_returned_with_fever  \
patient_id time_point_days                                                 
pa-001     3                                NaN                            
           3                                NaN                            
           3                                NaN                            
           3                                NaN                            
           3                                NaN                            

                            retreated when_retreated  
patient_id time_point_days                            
pa-001     3                      NaN                 
           3                      NaN                 
           3                      NaN                 
           3                      NaN                 
           3                      NaN

In [143]:
partial_format.to_csv('C:/Users/lzoeckler/Desktop/4plex/output_data/partially_formatted.csv')